This is a baseline for the HTCV Sem

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import shutil

from data_loader import load_data_entries
from constants import *

In [2]:
train_data_entries = load_data_entries(data_type="train",percentage_as_str="10")
print(len(train_data_entries))
print(train_data_entries)

280
[('socal-fire_00000862_post_disaster.png', 3), ('hurricane-michael_00000504_post_disaster.png', 1), ('hurricane-florence_00000500_post_disaster.png', 3), ('midwest-flooding_00000264_post_disaster.png', 1), ('santa-rosa-wildfire_00000368_post_disaster.png', 1), ('hurricane-florence_00000218_post_disaster.png', 0), ('hurricane-florence_00000435_post_disaster.png', 1), ('hurricane-michael_00000322_post_disaster.png', 1), ('midwest-flooding_00000426_post_disaster.png', 1), ('hurricane-matthew_00000219_post_disaster.png', 3), ('santa-rosa-wildfire_00000351_post_disaster.png', 2), ('socal-fire_00000563_post_disaster.png', 0), ('socal-fire_00000367_post_disaster.png', 4), ('santa-rosa-wildfire_00000283_post_disaster.png', 2), ('hurricane-florence_00000022_post_disaster.png', 3), ('hurricane-florence_00000436_post_disaster.png', 3), ('socal-fire_00001178_post_disaster.png', 0), ('socal-fire_00000886_post_disaster.png', 2), ('socal-fire_00000073_post_disaster.png', 0), ('midwest-flooding_00

In [3]:
def visualize_data(idx, data_type, data):
    img_name = data[idx][0]
    img_path = os.path.join(DATASET_FOLDER_PATH,data_type,"images",img_name)
    img = Image.open(img_path)
    img.show()

In [4]:

# Define the ResNet model
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ResNetClassifier, self).__init__()
        self.resnet = models.resnet18(weights=None)
        self.resnet.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.resnet(x)
        return x

# Create an instance of the ResNet classifier
num_classes = 5  # Number of output classes
model = ResNetClassifier(num_classes)

In [5]:
# Load and preprocess your dataset using torchvision.transforms
pre_norm_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the input image to match the expected size of ResNet (224x224)
    transforms.ToTensor()
])
# Calculate the mean and standard deviation of your dataset
path_to_dataset = os.path.join(DATASET_FOLDER_PATH,"train")
dataset = torchvision.datasets.ImageFolder(path_to_dataset, transform=pre_norm_transform)
data_point_names = np.array(train_data_entries)[:,0]

subset = [dataset[dataset.imgs.index((file_path, class_label))] for file_path, class_label in dataset.imgs if os.path.basename(file_path) in data_point_names]
loader = torch.utils.data.DataLoader(subset, batch_size=32, shuffle=False)

mean = 0.0
std = 0.0
total_samples = 0

for images, _ in loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    total_samples += batch_samples

mean /= total_samples
std /= total_samples

# Update the normalization transform using your dataset's mean and standard deviation
transform = transforms.Compose([
    pre_norm_transform,
    transforms.Normalize(mean, std)
])

In [7]:
print(mean)
print(std)

tensor([0.3048, 0.3366, 0.2518])
tensor([0.1172, 0.1034, 0.0987])


In [8]:
# Example dataset loading using torchvision.datasets.ImageFolder
dataset = torchvision.datasets.ImageFolder(path_to_dataset, transform=transform)
subset = [dataset[dataset.imgs.index((file_path, class_label))] for file_path, class_label in dataset.imgs if os.path.basename(file_path) in data_point_names]
dataloader = torch.utils.data.DataLoader(subset, batch_size=32, shuffle=True)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Save the trained model
torch.save(model.state_dict(), 'resnet_classifier.pth')


Epoch 1/10, Loss: 13.472020030021667
Epoch 2/10, Loss: 12.9510897397995
Epoch 3/10, Loss: 12.492850065231323
Epoch 4/10, Loss: 12.0121990442276
Epoch 5/10, Loss: 11.538281679153442
Epoch 6/10, Loss: 11.075330376625061
Epoch 7/10, Loss: 10.94474458694458
Epoch 8/10, Loss: 10.464341163635254
Epoch 9/10, Loss: 10.09627366065979
Epoch 10/10, Loss: 10.01226669549942


In [13]:
# Define the evaluation function
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# Create an instance of the ResNetClassifier
model = ResNetClassifier(num_classes)

# Load the saved model state dictionary
model.load_state_dict(torch.load('resnet_classifier.pth'))

# Set the device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the test dataset
test_dataset = torchvision.datasets.ImageFolder(root=os.path.join("..","dataset","test"), transform=transform)

# Create the test dataloader
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Move the model to the device
model.to(device)

# Evaluate the model
accuracy = evaluate_model(model, test_dataloader)

# Print the accuracy
print(f"Accuracy: {accuracy}")


Accuracy: 0.5273311897106109


first run:
Accuracy: 0.5273311897106109

In [ ]:
# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in dataloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")

# Save the trained model
torch.save(model.state_dict(), 'resnet_classifier_100epoch.pth')



In [ ]:
# Define the evaluation function
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode

    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# Create an instance of the ResNetClassifier
model = ResNetClassifier(num_classes)

# Load the saved model state dictionary
model.load_state_dict(torch.load('resnet_classifier_100epoch.pth'))

# Set the device for computation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create the test dataset
test_dataset = torchvision.datasets.ImageFolder(root=os.path.join("..","dataset","test"), transform=transform)

# Create the test dataloader
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Move the model to the device
model.to(device)

# Evaluate the model
accuracy = evaluate_model(model, test_dataloader)

# Print the accuracy
print(f"Accuracy: {accuracy}")
